In [1]:
import os
import urllib.request
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
import scipy
import statsmodels.api as sm
from sklearn.preprocessing import OrdinalEncoder

def freedman(data):
    # freedman method for choosing no of bins
    data = np.asarray(data, dtype=np.float_)
    IQR = scipy.stats.iqr(data, rng=(25, 75), scale=1, nan_policy="omit")
    N = data.size
    bw = (2 * IQR) / np.power(N, 1/3)
    mini= np.min(data)
    maxi=np.max(data)
    d = maxi-mini
    result = int((d / bw) + 1)
    return result

In [2]:
# loading all csv files using pandas
PATH = '../datasets/'
Study_A = pd.read_csv(PATH+"Study_A.csv")
Study_B = pd.read_csv(PATH+"Study_B.csv")
Study_C = pd.read_csv(PATH+"Study_C.csv")
Study_D = pd.read_csv(PATH+"Study_D.csv")
Study_E = pd.read_csv(PATH+"Study_E.csv") 

In [3]:
all_studies = pd.concat([Study_A,Study_B,Study_C,Study_D,Study_E]) 
# considering all given studies as we have data of PANSS_Score and visit days for all studies 

In [4]:
all_studies["P_total"] = np.sum(np.array([all_studies[f"P{i}"] for i in range(1,8)]),axis=0)
all_studies["N_total"] = np.sum(np.array([all_studies[f"N{i}"] for i in range(1,8)]),axis=0)
all_studies["G_total"] = np.sum(np.array([all_studies[f"G{i}"] for i in range(1,8)]),axis=0)

In [5]:
all_studies_control = all_studies[all_studies["TxGroup"]=="Control"]
all_studies_treatment = all_studies[all_studies["TxGroup"]!="Control"]
# grouping all studies A,B...E depending on the treatment or control groups

In [6]:
ordinal_encoder = OrdinalEncoder()
arr = np.array(all_studies["Country"]).reshape(-1,1)
arr1 = np.array(all_studies["TxGroup"]).reshape(-1,1)
encoded = ordinal_encoder.fit_transform(arr)
encoded1 = ordinal_encoder.fit_transform(arr1)
all_studies.loc[:,"Country"] = encoded
all_studies.loc[:,"TxGroup"] = encoded1

C:\Users\Hamesh\AppData\Local\Temp\ipykernel_14600\3013508270.py:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_studies.loc[:,"Country"] = encoded
C:\Users\Hamesh\AppData\Local\Temp\ipykernel_14600\3013508270.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  all_studies.loc[:,"TxGroup"] = encoded1


In [7]:
Study_A_grouped = []
same_id = Study_A.iloc[0,:]["PatientID"] # initialized id
dummy = []

dummy.append(Study_A.iloc[0,:])

for index,row in Study_A.iloc[1:,:].iterrows():
    if(row["PatientID"]!=same_id):
        Study_A_grouped.append(dummy)
        dummy=[]
        dummy.append(row)
        same_id = row["PatientID"]
    else:
        dummy.append(row)

### PATIENT SEGMENTATION

In [8]:
all_studies_treatment_initial = all_studies_treatment[all_studies_treatment["VisitDay"]==0] # rows with baseline measurement
all_studies_control_initial = all_studies_control[all_studies_control["VisitDay"]==0] 